In [9]:
import torch
import llm_transparency_tool.routes.graph
from llm_transparency_tool.models.tlens_model import TransformerLensTransparentLlm
from llm_transparency_tool.models.transparent_llm import TransparentLlm

from llm_transparency_tool.models.layerSkipCustom import LayerSkipLlamaTransparentLlm

model = LayerSkipLlamaTransparentLlm("facebook/layerskip-llama3.2-1B")


In [10]:
model.run("John is a good")

graph = llm_transparency_tool.routes.graph.build_full_graph(
    model,0,0.4
)

In [11]:
print(graph)
print(graph.edge_subgraph)
print(type(graph))

DiGraph with 245 nodes and 640 edges
<bound method Graph.edge_subgraph of <networkx.classes.digraph.DiGraph object at 0x10464d730>>
<class 'networkx.classes.digraph.DiGraph'>


In [12]:
import time
import torch

start_time = time.time()
output = model.run("John is a good")
torch.mps.synchronize()  # Ensures all MPS operations are complete
end_time = time.time()

inference_time = end_time - start_time
print(f"Inference time: {inference_time:.4f} seconds")


Inference time: 0.1205 seconds


In [13]:
print(output)

None


In [14]:
import time
import torch
from tqdm import tqdm

def time_inference(model, prompts, num_runs=10):
    times = []
    for _ in range(num_runs):
        for prompt in tqdm(prompts):
            start = time.time()
            output = model.run(prompt)
            torch.mps.synchronize()  # Critical for MPS accuracy
            end = time.time()
            times.append(end - start)
    return sum(times)/len(times)

mbpp_prompts = [
    "Write a function to find the minimum cost path to reach (m, n) from (0, 0) for the given cost matrix cost[][] and a position (m, n) in cost[][]."
    "Write a function to find the similar elements from the given two tuple lists.",
    "Write a python function to identify non-prime numbers.",
    "Write a function to find the largest integers from a given list of numbers using heap queue algorithm.",
    "Write a function to find the number of ways to fill it with 2 x 1 dominoes for the given 3 x n board.",
    "Write a python function to check whether the two numbers differ at one bit position only or not.",
    "Write a function to find all words which are at least 4 characters long in a string by using regex.",
    "Write a function to find squares of individual elements in a list using lambda function.",
    "Write a python function to find the minimum number of rotations required to get the same string.",
    "Write a function to get the n smallest items from a dataset.",
]

avg_time = time_inference(model, mbpp_prompts)
print(avg_time)


100%|██████████| 9/9 [00:00<00:00, 15.39it/s]

0.08310959074232313
